# Noise at Fixed Frequencies Simulation Environment

1. Load a clean signal acquired using GNURadio.
1. Add Noise at Fixed Frequencies to the ideal signal to simulate the behaviour of a blocking card.
1. Analyze the performance of the demodulator.

In [ ]:
from common_imports import *

# Useful in Windwos OS
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# PATHS
DATA_PATH     = r'../data/classic/'
RESULTS_PATH  = r'../data/results/'

# TO SHOW PLOTS
SHOW_PLOTS = True

### **1.** Load the signal of a clean communication

In [ ]:
# Paths
dataPath = DATA_PATH + 'classic_clean.raw'
expected_file = libnfc_file = DATA_PATH + 'expectedTxt/classic_clean.txt'

# Load data
data = load_mag(dataPath)

# Print the signal
plt.figure(figsize=(30, 4), color="#47B5FF"
plt.plot(data)
plt.show()

In [ ]:
# Load everything into a NfcSignal class
start = time()
s = NfcSignal(data, 
      expected_file = expected_file, 
      libnfc_file = libnfc_file,
      attack_mode = 0, 
      mean_samples = 0, 
      message_batch_size = 8)
end = time()
print(f"init duration {end-start}")

# Start Demodulation
s.start_demodulation()

Check demodulation stats on a clean signal

In [ ]:
s.demodulation_stats()

### **2.** Add noise at different fixed frequencies

In [ ]:
NOISE_TYPE_LIST = []
signals = []

DELTA_FREQ_START = 0.05e6
DELTA_FREQ_STOP  = 0.30e6
DELTA_FREQ_STEP  = 0.05e6

NFC_FREQ    = 13.56e6
Fs          = NFC_FREQ * 100
dt          = 1 / Fs
t           = np.arange(0, 1e-3, dt)
amp         = 0.0025

for delta_freq in np.arange(0.05e6,0.3e6,0.05e6):
    # Create base signal at 13.56MHz
    noise = amp * np.sin(2 * np.pi * (NFC_FREQ) * t)

    numIterations = int(((15e6 - 12e6)/delta_freq)/2.0)

    # Add other frequnecies
    for i in range(1,numIterations):
        noise += amp * np.sin(2 * np.pi * (NFC_FREQ + delta_freq * i) * t)
        noise += amp * np.sin(2 * np.pi * (NFC_FREQ + delta_freq * -1 *i) * t) 

    # Create the noisySignal
    resizedNoise = np.resize(noise, len(data))
    noisyData = resizedNoise + data

    # Add noisySignal to the list
    signals.append(noisyData)

    # Add name of noise to list
    NOISE_TYPE_LIST.append("delta_{}_kHz".format(delta_freq/10e4))

    print("\n💻 [NOISE CREATING = {} ]\n".format(("delta_noise_{}_kHz".format(delta_freq/10e4))))

    if(SHOW_PLOTS):
                # Plot FFT
        n = len(t)
        fhat = np.fft.fft(noise, n)                
        psd = fhat * np.conj(fhat) / n          
        freq = (1 / (dt * n)) * np.arange(n)
        idxs_half = np.arange(1, np.floor(n / 2), dtype=np.int32) 

        print("[📊 PLOT] : FFT")
        plt.figure(figsize=(4,4))
        plt.xlim(12.4e6,14.4e6)
        plt.xticks(ticks=np.arange(12.4e6,14.4e6,0.4e6), labels = ['{:,.2f}'.format(x) for x in np.arange(12.4,14.4,0.4)])
        plt.xlabel("Frequency [MHz]")
        plt.ylabel("Voltage [V]")
        plt.plot(freq[idxs_half], psd[idxs_half], color='#47B5FF')
        plt.tight_layout()
        plt.savefig(RESULTS_PATH + "plots/delta_noise_"+ str(delta_freq/10e4) + "_kHz.pdf", bbox_inches = 'tight', pad_inches = 0)
        plt.show()

#####  Analyze the performance of different signals

In [ ]:
resultsDF = pd.DataFrame(columns =  ['noise_type','reader_correct', 'reader_expected', 'tag_detected', 'tag_correct', 'tag_expected', 'total_correct', 'total_expected'])

In [ ]:
for i in range(len(signals)):
    print("\n💻 [SIGNAL = {} ]\n".format(NOISE_TYPE_LIST[i]))

    start = time()
    s = NfcSignal(signals[i], 
        expected_file = expected_file, 
        libnfc_file = libnfc_file,
        attack_mode = 0, 
        mean_samples = 0, 
        message_batch_size = 8)
    end = time()
    print(f"init duration {end-start}")

    # Start Demodulation
    s.start_demodulation()

    # Get demodulation stats and add blockig card blockingCardName
    print("📝 STATS: ")
    tmpDF = s.demodulation_stats()
    tmpDF['noise_type'] = NOISE_TYPE_LIST[i]

    # Add demodulation stats to the results DF
    resultsDF = pd.concat([resultsDF, tmpDF], ignore_index=True)

In [ ]:
resultsDF

##### Compute demodulation rates

In [ ]:
# Lists to store rates
readerDemodulationRateList = []
tagDemodulationRateList = []
tagDetectionRateList = []
totalDemodulationRateList = []

# Iterate through rows
for index, row in resultsDF.iterrows():
    readerDemodulationRateList.append(round(row['reader_correct']/row['reader_expected'],2))
    tagDetectionRateList.append(round(row['tag_detected']/row['tag_expected'],2))
    tagDemodulationRateList.append(round(row['tag_correct']/row['tag_expected'],2))
    totalDemodulationRateList.append(round(row['total_correct']/row['total_expected'],2))

# Add rates to resultsDF
resultsDF['reader_demodulation_rate'] = readerDemodulationRateList
resultsDF['tag_detection_rate'] = tagDetectionRateList
resultsDF['tag_demodulation_rate'] = tagDemodulationRateList
resultsDF['total_demodulation_rate'] = totalDemodulationRateList

### **4.** Save stats into CSV File

In [ ]:
# Save to CSV
resultsDF.to_csv(RESULTS_PATH + "delta_results.csv")  

In [ ]:
# Load from CSV
resultsDF = pd.read_csv(RESULTS_PATH + "delta_results.csv", index_col = 0) 
print(resultsDF)